# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do

**`Demstration of workshop content`***
- dlt hub:
  - https://dlthub.com
- Workshop video:
  - https://www.youtube.com/watch?v=oLXhBM7nf2Q
#
- Workshop notebook:
  - https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/cohorts/2024/workshops/dlt_resources/workshop.ipynb
#
- Intro:
  - https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/cohorts/2024/workshops/dlt_resources/data_ingestion_workshop.md


# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


#### **Question 1: What is the sum of the outputs of the generator for limit = 5?**

In [3]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

limit = 5
generator = square_root_generator(limit)

total = sum(generator)
print(f"The sum of the square roots is {total}.")

The sum of the square roots is 8.382332347441762.


#### **Question 2: What is the 13th number yielded**

In [5]:
limit = 13
generator = square_root_generator(limit)

thirteenth_number = None
for _ in range(limit):
    thirteenth_number = next(generator)

print(f"The 13th number is {thirteenth_number}.")

The 13th number is 3.605551275463989.


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [ ]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


#### ***Install & Import Dependencies***

In [6]:
!pip3 install dlt duckdb

  Using cached dlt-0.4.2-py3-none-any.whl.metadata (9.5 kB)
  Using cached duckdb-0.9.2-cp310-cp310-macosx_10_9_x86_64.whl.metadata (760 bytes)
  Using cached giturlparse-0.12.0-py2.py3-none-any.whl.metadata (4.5 kB)
  Using cached hexbytes-1.0.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached humanize-4.9.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached jsonpath_ng-1.6.1-py3-none-any.whl.metadata (18 kB)
  Using cached makefun-1.15.2-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached orjson-3.9.13-cp310-cp310-macosx_10_15_x86_64.macosx_11_0_arm64.macosx_10_15_universal2.whl.metadata (49 kB)
  Using cached pathvalidate-3.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached pendulum-3.0.0-cp310-cp310-macosx_10_12_x86_64.whl.metadata (6.9 kB)
  Using cached requirements_parser-0.5.0-py3-none-any.whl (18 kB)
  Using cached semver-3.0.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached setuptools-69.0.3-py3-none-any.whl.metadata (6.3 kB)
  Using cached simplejson-3.19.2-cp310-cp310

In [7]:
import dlt
import duckdb

#### 1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.

In [12]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

generator_1 = people_1()

 - Initialize db connection & pipe the data form the 1st generator:

In [13]:
# define the connection to load to.
generators_pipeline = dlt.pipeline(
                            destination  = 'duckdb',
                            dataset_name = 'people_container'
)


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(
                            data              = generator_1,
							table_name        = "people",
							write_disposition = "replace"
)

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.25 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_container
The duckdb destination used duckdb:////Users/sebastianwefers/Desktop/SEMESTER5/Data_Warehouse/Data_Engineering_Zoomcamp/WorkShop_DataIngestionAPIs/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707327891.6316001 is LOADED and contains no failed jobs


- Connect to the created db, display the corresponding table and determine the sum of all ages:

In [17]:
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
tables = conn.sql("show tables").df()
display(tables)

# let's see the data
print('Loaded data: ')
people = conn.sql("select * from people").df()
display(people)

# display the sum of all ages
print('Sum of ages: ')
total_ages = conn.sql("select sum(Age) total_ages from people").df()
display(total_ages)

Loaded tables: 


,name
0,_dlt_loads
1,_dlt_pipeline_state
2,_dlt_version
3,people


Loaded data: 


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707327891.6316001,9CpxkeRpEoSxyw
1,2,Person_2,27,City_A,1707327891.6316001,ZmYR6VgL+GnTgg
2,3,Person_3,28,City_A,1707327891.6316001,ADwzKZbO3QRn2Q
3,4,Person_4,29,City_A,1707327891.6316001,mLERUvIh71qZDg
4,5,Person_5,30,City_A,1707327891.6316001,T/1xlyW6chzPlw


Sum of ages: 


,total_ages
0,140.0


#### 2. Append the second generator to the same table as the first.

In [18]:
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

generator_2 = people_2()

In [19]:
# append the second generator to the same table
info2 = generators_pipeline.run(
                            data              = generator_2,
							table_name        = "people",
							write_disposition = "append"
)

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info2)

# let's see the data
print('Loaded data: ')
people = conn.sql("select * from people").df()
display(people)

Pipeline dlt_ipykernel_launcher load step completed in 0.25 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_container
The duckdb destination used duckdb:////Users/sebastianwefers/Desktop/SEMESTER5/Data_Warehouse/Data_Engineering_Zoomcamp/WorkShop_DataIngestionAPIs/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707327891.6316001 is LOADED and contains no failed jobs
Loaded data: 


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707327891.6316001,9CpxkeRpEoSxyw,None
1,2,Person_2,27,City_A,1707327891.6316001,ZmYR6VgL+GnTgg,None
2,3,Person_3,28,City_A,1707327891.6316001,ADwzKZbO3QRn2Q,None
3,4,Person_4,29,City_A,1707327891.6316001,mLERUvIh71qZDg,None
4,5,Person_5,30,City_A,1707327891.6316001,T/1xlyW6chzPlw,None
5,3,Person_3,33,City_B,1707328147.14367,ZvbM+WZVyllDCQ,Job_3
6,4,Person_4,34,City_B,1707328147.14367,qZ2fbKTdEPPzSQ,Job_4
7,5,Person_5,35,City_B,1707328147.14367,2/Rz12Lq3NbsSA,Job_5
8,6,Person_6,36,City_B,1707328147.14367,nCI2ovccSeCkjA,Job_6
9,7,Person_7,37,City_B,1707328147.14367,nhkq7FiZ+b8UzA,Job_7


Sum of ages: 


,total_ages
0,353.0


#### 3. **After correctly appending the data, calculate the sum of all ages of people.**

In [20]:
# display the sum of all ages
print('Sum of ages: ')
total_ages = conn.sql("select sum(Age) total_ages from people").df()
display(total_ages)

Sum of ages: 


,total_ages
0,353.0


# 3. Merge a generator
---
Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

#### ***`Question:`***
-  **Calculate the sum of ages of all the people loaded as described above.**
---

- Initialize new data set and connection:

In [31]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

generator_1 = people_1()
generator_2 = people_2()

generators_pipeline2 = dlt.pipeline(
                            destination  = 'duckdb',
                            dataset_name = 'peoples_to_be_merged'
)

# Load the first generator
info_1 = generators_pipeline2.run(
                            data              = generator_1,
                            table_name        = "peoples",
                            write_disposition = "replace"
)

# Load the second generator with 'merge' logic

info_2 = generators_pipeline2.run(
                            data              = generator_2,
                            table_name        = "peoples",
                            write_disposition = "merge",
                            primary_key       = "id"
)

# connect:
conn2 = duckdb.connect(f"{generators_pipeline2.pipeline_name}.duckdb")
# set the search path
conn2.sql(f"SET search_path = '{generators_pipeline2.dataset_name}'")

# Display the tables
print('Loaded tables: ')
table = conn2.sql("show tables").df()
display(table)

# Display the data
print("\n\n\n The merged peoples table:")
peoples = conn2.sql("SELECT * FROM peoples_to_be_merged.peoples").df()
display(peoples)

# Calculate the sum of ages
sum_of_ages    = conn2.sql("SELECT SUM(Age) FROM peoples_to_be_merged.peoples").fetchone()[0]
n_observations = conn2.sql("SELECT COUNT(*) FROM peoples_to_be_merged.peoples").fetchone()[0]
print(f"Solution\n{39*'~'}\nThe sum of all ages from {n_observations} people is: {sum_of_ages}.")

Loaded tables: 


,name
0,_dlt_loads
1,_dlt_pipeline_state
2,_dlt_version
3,peoples





 The merged peoples table:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707329397.5439308,dRXz9WAYdZ6YXQ,None
1,2,Person_2,27,City_A,1707329397.5439308,WjusyAktzFzN+g,None
2,4,Person_4,34,City_B,1707329398.161943,VQrfhjdBZsg26A,Job_4
3,7,Person_7,37,City_B,1707329398.161943,86dfIVArN6qJgQ,Job_7
4,5,Person_5,35,City_B,1707329398.161943,Ho15U6NEw4efGw,Job_5
5,8,Person_8,38,City_B,1707329398.161943,fpAtzFBrP9ni8g,Job_8
6,3,Person_3,33,City_B,1707329398.161943,Ut2TLba42peLDQ,Job_3
7,6,Person_6,36,City_B,1707329398.161943,s2DMoob156mWEg,Job_6


Solution
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The sum of all ages from 8 people is: 266.


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX

In [6]:
110_000/12

9166.666666666666